<a href="https://www.kaggle.com/luisprezsala/batman?scriptVersionId=89605270" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
pip install pysubgroup

ERROR: Could not find a version that satisfies the requirement pysubgroup (from versions: none)
ERROR: No matching distribution found for pysubgroup
Note: you may need to restart the kernel to use updated packages.
